In [4]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt


import os
import gc
import sys
srcpath = os.path.realpath('./../libraries')
sys.path.append(srcpath)

#import the basic libraries for Tabor AWT
from tevisainst import TEVisaInst
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from pulse_sequencev2 import EPR_pulse
import pyte_visa_utils as pyte

# import the libraries for the SignalCore unit
import sc5318a
import time
from sc5318a import *

import signalcore
from signalcore import *


In [ ]:
# Initializing all components
def AWT_init(inst_addr='169.254.18.140',sampleRateDAC = 1e9, paranoia_level=2):
    ## Tabor AWT
    # The paranoia-level is either low (0), normal (1) or debug (2)
    inst = TEVisaInst(inst_addr)
    resp = inst.send_scpi_query("*IDN?")
    print('connected to: ' + resp)
    # Reset the instrument
    inst.send_scpi_cmd('*CLS; *RST')
    print ('Reset instrument ..')
    dac_mode = 16 
    model_name = inst.send_scpi_query('SYST:INF:MODel?') # Get the model
    print('Model: {0} '.format(model_name))
    # Get model dependant parameters:
    # scaling
    max_dac = 2 ** dac_mode - 1
    half_dac = max_dac / 2.0
    wpt_type = np.uint16   
    # Get the available memory in bytes of wavform-data (per DDR):
    resp = inst.send_scpi_query(":TRACe:FREE?")
    arbmem_capacity = int(resp)
    print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))
    cmd = ':FREQ:RAST {0}'.format(sampleRateDAC) 
    inst.send_scpi_cmd(cmd)
    print('Sample rate set to: {0} Hz'.format(sampleRateDAC))
    
    print('Tabor AWT initialized')
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)
    return inst

def signalcore_init(address='1000491B'):
    """
    Initialize the SignalCore device.
    """
    SC1 = SignalCore_SC5318A("SC1", address)
    signalcore.__dict__.update({'SC1':SC1}) # update the global variables to signalcore
    print("connected to: SignalCore SC5318A" )
    get_idn(SC1)
    print("Device current temperature:{} degree".format(device_temp()))

    print ('Device current RF parameters:')
    display_rf_parameters()

    #get the gain conversion
    print ('gain conversion: {}'.format(calc_gain()))

    #reading from the eeprom
    #read_eeprom(0, 0x04, 4)

    #example of the reg_read and reg_write commands
    # #reads back the device's serial number
    # reg_read(0x30, 0)
    #sets RF in freq to 1200000000000 Hz
    reg_write(0x10, 1200000000000)

    print("Changing the IF frequency value...")
    set_if_frequency(1.2e9)
    get_if_atten_value()

    print("Closing device")
    close_device()

AWT_init()
signalcore_init('1000491B')




connected to: Tabor Electronics,P2582D,000002248484,1.237.0 --slot#: 5, 
Reset instrument ..
Model: P2582D 
Available memory per DDR: 4,286,532,544 wave-bytes
Sample rate set to: 1000000000.0 Hz
Tabor AWT initialized
0, no error
connected to: SignalCore SC5318A
{'vendor': 'SignalCore', 'model': 'SC5318A', 'serial_number': '1000491B', 'firmware_revision': 1.5099999904632568, 'hardware_revision': 5.0, 'device_interface': 2, 'calibration_date': '24-8-15', 'manufacture_year': '2025', 'manufacture_month': '4', 'manufacture_day': '22'}
47.0
Device current temperature:47.0 degree
Device current RF parameters:
{'vendor': 'SignalCore', 'model': 'SC5318A', 'rf_frequency': 1200000000.0, 'if_frequency': 1200000000.0, 'lo_frequency': 0.0, 'rf_atten_value': 0.0, 'if_atten_value': 9.0, 'bypass_conversion': 0, 'rf_amp_enable': 0, 'if_out_enable': 1, 'invert_spectrum': 0}
-0.9058429598808289
gain conversion: -0.9058429598808289
Changing the IF frequency value...
The IF Attenuation is : 9.00
Closing dev

In [4]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "get_ipython().run_line_magic('matplotlib', 'notebook')\nimport numpy as np\nimport time\nimport ipywidgets as widgets\nimport matplotlib.pyplot as plt\n\n\nimport os\nimport gc\nimport sys\nsrcpath = os.path.realpath('./../libraries')\nsys.path.append(srcpath)\n\n#import the basic libraries for Tabor AWT\nfrom tevisainst import TEVisaInst\nfrom teproteus import TEProteusAdmin as TepAdmin\nfrom teproteus import TEProteusInst as TepInst\nfrom pulse_sequencev2 import EPR_pulse\nimport pyte_visa_utils as pyte\n\n# import the libraries for the SignalCore unit\nimport sc5318a\nimport time\nfrom sc5318a import *\n\nimport signalcore\nfrom signalcore import *",
  '# Initializing all components\ndef AWT_init(inst_addr=\

# Testing YAML config file

Document initializing the configuration of the EPR spectrometer. 
The CHANNELS section reflects the physical connections between the devices for documentation, which are defined in the second section. 
 
2-channel configuration for ELDOR using a VDI synthesizer via a mixer for probing and a Windfreak synthesizer for pumping without mixer, both synthesizer output sent to the same AMC via a rapid SP2T switch 
________________________________ 
MODES 
 
#1 – Pulsed with detection 
Synthesizer		VDIS217 
AMC			AMC629-16 
Mix			AO1@TaborAWT 
Power			DO1@ArduinoUCA 
Switch			DO1@TaborAWT 
Pulsing IF		AO1@TaborAWT, 500e6 
Detection LO		AO2@TaborAWT, 0.444e6 
Detection LO gain	LOgain@SignalCore 
Detection RF gain	RFgain@SignalCore 
 
#2 – CW without detection 
Synthesizer#		Windfreak 
AMC			AMC629-16 
Switch			DO2@TaborAWT	 
 
________________________________ 
DEVICES 
 
VDIS217 
FTCI address		VDIS217 
Min freq			8 
Max freq			18 
 
Windfreak 
Serial address		COMXXX 
Min freq			0.05 
Max freq			15 
Max power		15 
 
% AMC does not interface with the code. values for validating purpose 
AMC629-16 
Multiplication factor	16 % for calculating synth freq
Min freq			190 % validation
Max freq			196 
% Max input power		12 
Power-UCA table		<path to data> 
 
AMC629-8 
Multiplication factor	16 
Min freq			95 
Max freq			98 
Power-UCA table		<path to data> 
 
TaborAWT 
FTCI address		<address> 
AWG bandwidth		2.5 
Digitizer bandwidth	2.5 
Memory			XXX 
 
SignalCore 
Serial address		COMXXX 
 
 
ArduinoUCA 
Serial address		COMXXX 

In [ ]:
'''